In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

try:
    import ipywidgets as widgets
    from IPython.display import display
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False


In [2]:
def load_head3_jsonl(head3_path: str):
    """
    Returns:
      frames: sorted list of frame ids available in head3
      head3_by_frame: dict[frame_id] -> record (weights, means, covariances, torso_component, etc.)
    """
    head3_by_frame = {}
    with open(head3_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rec = json.loads(line)
            # Expect rec to contain 'frame' key (adapt if your key differs)
            fid = int(rec["frame"])
            head3_by_frame[fid] = rec

    frames = sorted(head3_by_frame.keys())
    return frames, head3_by_frame


In [3]:
def plot_cov_ellipse(ax, mean2, cov2, n_std=2.0, **kwargs):
    """
    Draws an ellipse for a 2D Gaussian covariance matrix.
    - mean2: (2,)
    - cov2: (2,2)
    - n_std: how many standard deviations (2 ~ 95% mass for Gaussian)
    """
    # Eigen-decomposition
    vals, vecs = np.linalg.eigh(cov2)
    # Sort eigenvalues descending
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    # Angle of major axis
    theta = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))

    # Width/height are 2* n_std * sqrt(eigenvalues)
    width, height = 2 * n_std * np.sqrt(np.maximum(vals, 1e-12))

    from matplotlib.patches import Ellipse
    ell = Ellipse(xy=mean2, width=width, height=height, angle=theta, fill=False, **kwargs)
    ax.add_patch(ell)
    return ell


In [4]:
def select_plane_indices(plane: str):
    """
    plane in {"xy","yz","xz"}
    Returns indices into [x,y,z] => (i,j)
    """
    plane = plane.lower()
    if plane == "xy":
        return 0, 1
    if plane == "yz":
        return 1, 2
    if plane == "xz":
        return 0, 2
    raise ValueError("plane must be one of: 'xy', 'yz', 'xz'")


In [5]:
def visualize_gmm_clouds(
    head1_csv: str,
    head3_jsonl: str,
    plane: str = "xy",
    n_std: float = 2.0,
    p_torso_thr: float = None,   # if set, only plot points with p_torso >= thr
    max_points: int = 4000       # for safety if a frame has many points
):
    """
    Visualize per-frame Gaussian components (K=2) as covariance ellipses + scatter points.
    - plane: "xy" / "yz" / "xz"
    - n_std: ellipse size (2.0 is common)
    - p_torso_thr: if not None, filter points by p_torso
    """

    head1_df = pd.read_csv(head1_csv)
    frames3, head3_by_frame = load_head3_jsonl(head3_jsonl)

    # Basic sanity checks
    required_cols = {"frame", "x", "y", "z"}
    if not required_cols.issubset(set(head1_df.columns)):
        raise ValueError(f"Head-1 CSV must contain columns: {required_cols}, found: {head1_df.columns.tolist()}")

    if "p_torso" not in head1_df.columns:
        raise ValueError("Head-1 CSV must contain 'p_torso' column for this visualization.")

    i, j = select_plane_indices(plane)

    def _plot_frame(fid: int):
        if fid not in head3_by_frame:
            print(f"[WARN] frame {fid} not in Head-3 JSONL")
            return

        rec = head3_by_frame[fid]

        # Expect these keys (adapt if your JSON uses different names)
        # weights_pi: shape (K,)
        # means_mu_xyz: shape (K,3)
        # covariances_Sigma: shape (K,3,3)
        weights = np.array(rec["weights_pi"], dtype=float)
        means = np.array(rec["means_mu_xyz"], dtype=float)
        covs   = np.array(rec["covariances_Sigma"], dtype=float)
        torso_comp = int(rec.get("torso_component", -1))

        # Points for this frame from Head-1
        pts = head1_df[head1_df["frame"] == fid].copy()
        if pts.empty:
            print(f"[INFO] No points in Head-1 for frame {fid} (likely gated out).")
            return

        if p_torso_thr is not None:
            pts = pts[pts["p_torso"] >= p_torso_thr]

        if pts.empty:
            print(f"[INFO] After p_torso filter, no points left in frame {fid}.")
            return

        # Safety sampling for plotting speed (does NOT change stored data)
        if len(pts) > max_points:
            pts = pts.sample(max_points, random_state=0)

        coords = pts[["x","y","z"]].to_numpy()
        xy = coords[:, [i, j]]
        cvals = pts["p_torso"].to_numpy()

        fig, ax = plt.subplots(figsize=(7.5, 6))

        sc = ax.scatter(xy[:,0], xy[:,1], c=cvals, s=10)
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label("p_torso")

        # Draw Gaussian ellipses for each component
        for k in range(means.shape[0]):
            mean2 = means[k, [i, j]]
            cov2  = covs[k][np.ix_([i, j], [i, j])]

            # line style highlight torso component
            if k == torso_comp:
                lw = 2.5
                ls = "-"
            else:
                lw = 1.5
                ls = "--"

            plot_cov_ellipse(
                ax,
                mean2=mean2,
                cov2=cov2,
                n_std=n_std,
                linewidth=lw,
                linestyle=ls
            )

            ax.scatter(mean2[0], mean2[1], s=120, marker="x")

            # Text label: component + weight
            ax.text(
                mean2[0], mean2[1],
                f" k={k}, π={weights[k]:.2f}" + (" (torso)" if k == torso_comp else ""),
                fontsize=10
            )

        ax.set_title(f"GMM Clouds (plane={plane.upper()}) | frame={fid}")
        ax.set_xlabel(plane[0].upper())
        ax.set_ylabel(plane[1].upper())
        ax.grid(True)
        plt.show()

    if not HAS_WIDGETS:
        print("ipywidgets not available. Plotting first available frame only.")
        _plot_frame(frames3[0] if frames3 else 1)
        return

    slider = widgets.IntSlider(
        value=frames3[0] if frames3 else 1,
        min=min(frames3) if frames3 else 1,
        max=max(frames3) if frames3 else 1,
        step=1,
        description="frame",
        continuous_update=False
    )

    # When slider changes, plot frame
    out = widgets.Output()

    def on_change(change):
        if change["name"] == "value":
            with out:
                out.clear_output(wait=True)
                _plot_frame(change["new"])

    slider.observe(on_change, names="value")

    display(slider, out)

    # initial plot
    with out:
        _plot_frame(slider.value)


In [ ]:
head1_csv = r"E:\0.TA_Teguh\GMM\Head 1\B\Jalan53.csv"
head3_jsonl = r"E:\0.TA_Teguh\GMM\Head 3\B\Jalan53.jsonl"

visualize_gmm_clouds(
    head1_csv=head1_csv,
    head3_jsonl=head3_jsonl,
    plane="yz",        # coba "yz" dan "xz" juga
    n_std=2.0,         # 2 sigma ellipse
    p_torso_thr=None,  # atau set 0.2/0.5 untuk lihat torso dominan
    max_points=3000
)


IntSlider(value=1, continuous_update=False, description='frame', max=160, min=1)

Output()